In [1]:
# Importing dependencies

import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Loading the dataset as dataframe

df = pd.read_csv('dataset.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [3]:
# Viewing the number of rows and columns in the data frame

df.shape

(4803, 24)

In [4]:
# Viewing the data-types for each feature

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [5]:
# Selecting relevant features from the dataset

selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director', 'spoken_languages', 'vote_average']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director', 'spoken_languages', 'vote_average']


In [6]:
# Converting relevant features' Dtype to string type

df['genres'] = df['genres'].astype(str)
df['keywords'] = df['keywords'].astype(str)
df['tagline'] = df['tagline'].astype(str)
df['cast'] = df['cast'].astype(str)
df['director'] = df['director'].astype(str)
df['spoken_languages'] = df['spoken_languages'].astype(str)
df['vote_average'] = df['vote_average'].astype(str)

In [7]:
# Viewing updated Dtypes

df[selected_features].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   genres            4803 non-null   object
 1   keywords          4803 non-null   object
 2   tagline           4803 non-null   object
 3   cast              4803 non-null   object
 4   director          4803 non-null   object
 5   spoken_languages  4803 non-null   object
 6   vote_average      4803 non-null   object
dtypes: object(7)
memory usage: 262.8+ KB


In [8]:
# Replacing the null valuess with null string

for feature in selected_features:
  df[feature] = df[feature].fillna('')

In [9]:
# Combining all the 7 selected features

combined_features = df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['cast']+' '+df['director']+' '+df['spoken_languages']+' '+df['vote_average']
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance nan A newlywed couple's honeymo...
4800    Comedy Drama Romance TV Movie date love at fir...
4801    nan nan A New Yorker in Shanghai Daniel Henney...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [10]:
# Converting the text data to feature vectors

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)
print(feature_vectors)

  (0, 16039)	0.11439570003427277
  (0, 5042)	0.11439570003427277
  (0, 5032)	0.11439570003427277
  (0, 4955)	0.03382535004447686
  (0, 10828)	0.06445816090212846
  (0, 4928)	0.03382535004447686
  (0, 7699)	0.06445816090212846
  (0, 2443)	0.16836329692063867
  (0, 7796)	0.10995558747551541
  (0, 13085)	0.18933226900260217
  (0, 10276)	0.156532471800267
  (0, 8801)	0.22135674843150127
  (0, 14680)	0.14768159042461354
  (0, 16854)	0.19342275408321374
  (0, 14130)	0.2007609503375522
  (0, 13382)	0.21224953183557232
  (0, 17481)	0.19687970082852738
  (0, 17195)	0.23046396502743027
  (0, 13412)	0.1464201794631719
  (0, 11558)	0.2652429833464349
  (0, 11246)	0.08820849062413907
  (0, 17186)	0.12497561131298904
  (0, 15338)	0.06916683023277362
  (0, 4967)	0.23419264929303912
  (0, 14340)	0.20852084756996345
  :	:
  (4801, 4955)	0.03617069863845458
  (4801, 10828)	0.03446374848618309
  (4801, 4928)	0.03617069863845458
  (4801, 7699)	0.03446374848618309
  (4802, 7029)	0.5668142426691929
  (4802,

In [11]:
# Similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.0760085  0.09322493 ... 0.0067335  0.00688991 0.00656662]
 [0.0760085  1.         0.04073079 ... 0.03956106 0.00417287 0.00397707]
 [0.09322493 0.04073079 1.         ... 0.0126836  0.06129651 0.01236927]
 ...
 [0.0067335  0.03956106 0.0126836  ... 1.         0.01914294 0.03765868]
 [0.00688991 0.00417287 0.06129651 ... 0.01914294 1.         0.01866852]
 [0.00656662 0.00397707 0.01236927 ... 0.03765868 0.01866852 1.        ]]


In [12]:
# Verifying if the data is whole or not

print(similarity.shape)

(4803, 4803)


In [13]:
# Input from the user

user_input = input(' Enter your favourite movie name : ')
print(user_input)

Avengers


In [14]:
# Creating a list of titles in the dataset

title_list = df['title'].tolist()
print(title_list)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [15]:
# Finding present matches in dataset

present_matches = difflib.get_close_matches(user_input, title_list)
print(present_matches)

if present_matches:
    close_match = present_matches[0]
    print(close_match)
else:
    print("No matches found")

['The Avengers', 'Swingers']
The Avengers


In [16]:
# Finding the index of the movie

index_of_the_movie = df[df.title == close_match]['index'].values[0]
print(index_of_the_movie)

16


In [17]:
# Getting the list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.03964147146124955), (1, 0.022508993470531032), (2, 0.04746684562557301), (3, 0.01145564380849962), (4, 0.06123536989269224), (5, 0.02036133556532417), (6, 0.027914814812930087), (7, 0.7870301215992542), (8, 0.01253015348077051), (9, 0.16571939889697251), (10, 0.08165044873596168), (11, 0.021856957912970827), (12, 0.018628798173312804), (13, 0.016618097010591453), (14, 0.18728391802557282), (15, 0.049626627455823885), (16, 1.0), (17, 0.01920666572395907), (18, 0.029372381231094626), (19, 0.015636115650843274), (20, 0.11602824128825993), (21, 0.028953892380337617), (22, 0.01096300481426188), (23, 0.04118754047585431), (24, 0.01700477024334122), (25, 0.021391398980252732), (26, 0.3422153918210615), (27, 0.0324949373101697), (28, 0.07295026605873868), (29, 0.029946254334410644), (30, 0.10729757308047902), (31, 0.21825633082337656), (32, 0.033799287698099), (33, 0.2538058758167192), (34, 0.004440488100981495), (35, 0.04163102066233787), (36, 0.054100500316463795), (37, 0.012772430901

In [18]:
# Checking on the length of similarity score

len(similarity_score)

4803

In [19]:
# Sorting the movies based on their similarity score in descending order

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(16, 1.0), (7, 0.7870301215992542), (85, 0.43288486215072636), (79, 0.3678720130903796), (26, 0.3422153918210615), (126, 0.3155791981302727), (511, 0.2924077784429926), (174, 0.2803714612246722), (64, 0.27787387301670213), (182, 0.275824300464589), (129, 0.27251723184035187), (203, 0.2555216512731597), (33, 0.2538058758167192), (788, 0.2467349980409967), (46, 0.2432669295686404), (38, 0.2374003172097894), (4759, 0.2331127065180468), (169, 0.23116380598933306), (31, 0.21825633082337656), (14, 0.18728391802557282), (870, 0.18568577313231405), (782, 0.17298782822235784), (68, 0.17122791372393312), (94, 0.16667545172234238), (9, 0.16571939889697251), (101, 0.16435100563435362), (1192, 0.1550889790961475), (241, 0.15460429723752644), (1294, 0.15094087022200792), (2197, 0.15022469865273186), (1720, 0.14931852056662007), (713, 0.14859964655049454), (122, 0.1481406427195412), (1740, 0.14813108423902493), (242, 0.14515763744866095), (700, 0.1412307839155349), (421, 0.1406974781130244), (2390, 

In [20]:
# Printing the sorted list of movies

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Captain America: The Winter Soldier
4 . Iron Man 2
5 . Captain America: Civil War
6 . Thor: The Dark World
7 . X-Men
8 . The Incredible Hulk
9 . X-Men: Apocalypse
10 . Ant-Man
11 . Thor
12 . X2
13 . X-Men: The Last Stand
14 . Deadpool
15 . X-Men: Days of Future Past
16 . The Amazing Spider-Man 2
17 . The Image Revolution
18 . Captain America: The First Avenger
19 . Iron Man 3
20 . Man of Steel
21 . Superman II
22 . The Spirit
23 . Iron Man
24 . Guardians of the Galaxy
25 . Batman v Superman: Dawn of Justice
26 . X-Men: First Class
27 . Spawn
28 . Teenage Mutant Ninja Turtles: Out of the Shadows
29 . Serenity


In [21]:
# Putting together the recommendation model in a single cell

user_input = input(' Enter your favourite movie name : ')

title_list = df['title'].tolist()

present_matches = difflib.get_close_matches(user_input, title_list)

close_match = present_matches[0]

index_of_the_movie = df[df.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Captain America: The Winter Soldier
4 . Iron Man 2
5 . Captain America: Civil War
6 . Thor: The Dark World
7 . X-Men
8 . The Incredible Hulk
9 . X-Men: Apocalypse
10 . Ant-Man
11 . Thor
12 . X2
13 . X-Men: The Last Stand
14 . Deadpool
15 . X-Men: Days of Future Past
16 . The Amazing Spider-Man 2
17 . The Image Revolution
18 . Captain America: The First Avenger
19 . Iron Man 3
20 . Man of Steel
21 . Superman II
22 . The Spirit
23 . Iron Man
24 . Guardians of the Galaxy
25 . Batman v Superman: Dawn of Justice
26 . X-Men: First Class
27 . Spawn
28 . Teenage Mutant Ninja Turtles: Out of the Shadows
29 . Serenity
